# Simple EDA data video game sales

## Info

This dataset contains a list of video games with sales greater than 100,000 copies. It was generated by a scrape of vgchartz.com.

Fields include

    Rank - Ranking of overall sales
    Name - The games name
    Platform - Platform of the games release (i.e. PC,PS4, etc.)
    Year - Year of the game's release
    Genre - Genre of the game
    Publisher - Publisher of the game
    NA_Sales - Sales in North America (in millions)
    EU_Sales - Sales in Europe (in millions)
    JP_Sales - Sales in Japan (in millions)
    Other_Sales - Sales in the rest of the world (in millions)
    Global_Sales - Total worldwide sales.

The script to scrape the data is available at https://github.com/GregorUT/vgchartzScrape.
It is based on BeautifulSoup using Python.
There are 16,598 records. 2 records were dropped due to incomplete information.

Acesso: 03/07/2020
Link:https://www.kaggle.com/gregorut/videogamesales

In [ ]:

#Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns; sns.set()



In [ ]:

#importing data
data = pd.read_csv('../input/videogamesales/vgsales.csv')

In [ ]:
# 5 first lines
data.head()

In [ ]:
# simple info about data
data.info()

In [ ]:
# Looking values nan
data.isna().sum()

In [ ]:
# Looking column "Year"

data['Year']


Abaixo é criada uma nova coluna chamada year em que contém as datas, anos, no formato datetime. 

In [ ]:

# converte os valores, Float64, da coluna "Year" para datetime 
def to_formatData(x):
    if(not np.isnan(x)):
        x = int(x)
        date = pd.to_datetime(x, yearfirst=True, format="%Y")
        return date

# criando nova coluna "year" com valores de "Year" convertidos para datetime
data['year'] = data['Year'].apply(to_formatData)

In [ ]:

# new data
data.head()

In [ ]:

# Imprimindo o primeiro e último ano de registro (intervalo de anos)

print('Data mínima: ', data['year'].min())
print('Data maxíma: ', data['year'].max())

In [ ]:

# verificando coluna "Name"

#data['Name'].unique().shape
data['Name'].value_counts()

Podemos ver que alguns títulos aparecem mais de uma vez.

In [ ]:
data[data['Name'] == data['Name'].value_counts().index[0]] # imprimindo linhas em que "Name" = "Need for Speed: Most Wanted", para verificar os registros com mesmo "Name" 

Podemos ver, aparentemente, que o motivo de alguns títulos se repetirem é devido a terem sido lançados em plataformas diferentes.

In [ ]:

data['Platform'].value_counts().plot.bar(figsize=(12,3))
g = plt.gca()
g.set_title('Count Plataform rank: 1980 até 2020', size=14)
g.set_ylabel('Count', size=14)
g.set_xlabel('Platform', size=14)
plt.show()

Isso nos mostra que entre 1980 e 2020 o <b>DS</b> foi a plataforma que apresentou maior número de lançamentos.

In [ ]:

data['Publisher'].value_counts(normalize=True)[0:30].plot.bar(figsize=(12,3)) # 30 maiores publicadoras dos games
g = plt.gca()
g.set_title('Count greather Publisher: 1980 até 2020', size=14)
g.set_xlabel('Publisher', size=14)
g.set_ylabel('Frequency', size=14)
plt.show()

Podemos ver que entre 1980 e 2020 a <b>Electronic Arts</b> apresentou maior número de lançamentos, considerando plataformas diferentes.

In [ ]:

data['Genre'].value_counts().plot.bar()
g = plt.gca()
g.set_title('Count Genre title')
g.set_xlabel('Genre')
g.set_ylabel('Count')
plt.show()


Podemos observar, considerando os lançamentos, que entre 1980 e 2020 ocorreu maior número de lançamentos de jogos do gênero <b>Action</b>.

Abaixo criamos um novo dataFrame em que é deletado os registros que apresentam valores Nan e é colocado o ano como index em cada registro.

In [ ]:

data2 = data.dropna() # apagando registros com Nan
data2 = data2.set_index(data2['year']);  # colocando index como a coluna 'year'
data2.drop(['Year','year'], inplace=True, axis=1) # deletando colunas 'Year' e 'year'

In [ ]:
#data2.index.isna().sum()
data2.info()

In [ ]:
data2.head()

In [ ]:

# realizando uma soma de todos os registros que apresentam o mesmo ano, dessa forma
# é possível verificar o total das vendas 'Global_Sales' por cada ano, abaixo é realiado uma plotagem
# do compartamento de 'Global_Sales' durante os anos.

globalSalesPerYear = data2['Global_Sales'].resample('Y').sum()
globalSalesPerYear

In [ ]:
globalSalesPerYear.plot(figsize=(12,4))
g = plt.gca()
g.set_ylabel('Total Global Sales')
g.set_title('Total Global Sales per year')
plt.show()

Com base nas cécluas acima podemos observar que existe um pico nas vendas globais no ano de 2008, sendo que depois desse ano os valores caem rapídamente. 

In [ ]:
data2[data2.index >= pd.to_datetime('2017')] # pegando todos os registros a partir do ano de 2017

É possível observar que a partir do ano de 2017 quase não é apresentado registros, o que pode revelar que o nosso conjunto de dados apresenta algum problema, podendo ter sido coletado de forma incorreta ou incompleta ou a partir de uma fonte viesada. Considerando isso, continuamos com outras informações que poderiamos querer a partir desse conjunto de dados.

Podemos pegar o título com maior venda em cada ano fazendo:

In [ ]:

# considerando o ano de 2010, por exemplo

#m = data2[data2.index == pd.to_datetime('2006-01-01')]
#m[m['Global_Sales'] == m['Global_Sales'].max()]


# Intervalo de anos dentro dos dados. freq = 'YS' significa frequencia em anos, pegano o inicio do ano
# 'yearStart'. Experimente usar apenas freq = 'Y' para ver o resultado. Mais sobre esses detalhes em [2] e [3]
dateRange = pd.date_range('1980-01-01','2020-01-01', freq='YS'); #dateRange

In [ ]:

# Títulos com maior venda em cada ano

MVendaPerYear = pd.DataFrame(columns=data2.columns)

for y in dateRange:
    temp = data2[data2.index == y] # seleciona os registros por ano
    MVendaPerYear = pd.concat([MVendaPerYear, temp[temp['Global_Sales'] == temp['Global_Sales'].max()]])
    
MVendaPerYear

In [ ]:

# resumindo dados acima
sns.catplot(x='Name', y='Global_Sales', data=MVendaPerYear, height=5, aspect=4, kind='bar')
g = plt.gca()
g.set_title('Títulos com maiores vendas de cada ano (1980 - 2020)', size=14)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.catplot(x='Genre', y='Global_Sales', data=data2, height=4, aspect=3)
g = plt.gca()
g.set_title('Global Sales per Genre')

In [ ]:
sns.catplot(x='Platform', y='Global_Sales', data=data2, height=4, aspect=3)
g = plt.gca()
g.set_title('Global Sales per Platform')

# References


[1]https://www.kaggle.com/gregorut/videogamesales
<br>
[2]https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html
<br>
[3]https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html
<br>
[4]https://www.drawingfromdata.com/how-to-rotate-axis-labels-in-seaborn-and-matplotlib
<br>
[5]https://www.drawingfromdata.com/setting-figure-size-using-seaborn-and-matplotlib
<br>